In [7]:
import cv2
from skimage import io, img_as_float
from skimage.filters import rank
from skimage.morphology import disk
import numpy as np
# Lista para almacenar los resultados de detección de rostro (1 si es un rostro, 0 si no lo es)
face_results = []
keyp=[]
# Crear un objeto SIFT
sift = cv2.xfeatures2d.SIFT_create()

def detect_faces(r):
    image = cv2.imread(r)
    # Detectar los puntos clave y calcular los descriptores de características
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # Verificar si se detectaron puntos clave (rostros)
    keyp=len(keypoints)
    return keyp

def evaluate_resolucionH(a):
    #print(a)
    img = cv2.imread(a)
    # Verificar la resolución de la imagen
    height, width, channels = img.shape
    return height

def evaluate_resolucionW(a):
    #print(a)
    img = cv2.imread(a)
    # Verificar la resolución de la imagen
    height, width, channels = img.shape
    return width

def evaluate_brillo(a):
    #print(a)
    img = cv2.imread(a)
    # Calcular la iluminación promedio de la imagen
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    mean_brightness = v.mean()
    #print("Falla brillo")
    #print(mean_brightness)
    return mean_brightness

def evaluate_contraste(a):
    #print(a)
    img = cv2.imread(a)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mean_contrast = cv2.Laplacian(gray, cv2.CV_64F).var()
    #print("mean contraste")
    #print(mean_contrast)
    #print("Falla contraste")
    return mean_contrast

def evaluate_entropia(a):
    im1 = img_as_float(io.imread(a, as_gray=True))
    radius = 15
    selem = disk(radius)
    entropy = rank.entropy(im1, selem=selem)
    mean_entropy = np.mean(entropy)
    #print("Falla entropia")
    #print(mean_entropy)
    return mean_entropy

def difuminacion(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    return laplacian_var

In [8]:
import cv2
from skimage import io, img_as_float
from skimage.filters import rank
from skimage.morphology import disk
import numpy as np
lista_fallas=[]
imagenes=[]
carpeta_base1=[]
n1=[]
puntosdet=[]
brillo=[]
contr=[]
reso=[]
entro=[]
difu=[]
####
def evaluate_image_qualitys(a):
    #carpet1, archi1 = os.path.split(a)
    #c1= os.path.basename(carpet1)
    if evaluate_resolucionH(a) < 64 or evaluate_resolucionW(a)< 64:
        return False
    if evaluate_brillo(a) < 14:
        return False
    if evaluate_contraste(a) < 3.5:
        return False
    if evaluate_entropia(a) < 3:
        return False
    if detect_faces(a)<=3:
        return False
    if  difuminacion(a) < 5.7:
        return False
    # Si todos los criterios se cumplen, la imagen es de buena calidad
    return True


In [9]:
import cv2
import skimage
from skimage.feature import hog
from sklearn.metrics.pairwise import cosine_similarity


def sim_coseno(ruta_imagen1, ruta_imagen2):
    image1 = skimage.io.imread(ruta_imagen1)
    image2 = skimage.io.imread(ruta_imagen2)

    # Obtener los descriptores de características HOG
    fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
                          cells_per_block=(1, 1), visualize=True, multichannel=True, channel_axis=2)
    fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
                          cells_per_block=(1, 1), visualize=True, multichannel=True, channel_axis=2)

    # Ajustar las matrices a la misma dimensión
    max_dim = max(fd1.shape[0], fd2.shape[0])
    fd1 = np.resize(fd1, (max_dim,))
    fd2 = np.resize(fd2, (max_dim,))

    # Calcular la similitud del coseno entre los descriptores
    similitud = cosine_similarity(fd1.reshape(1, -1), fd2.reshape(1, -1))

    return similitud[0][0]


In [10]:
import os
import pandas as pd
import numpy as np
#directorio_inicial1 = 'D:\sentirobots\Datasets\AFEW\Datos\Train_AFEW\AlignedFaces_LBPTOP_Points\AlignedFaces_LBPTOP_Points\Ejercicio entropia' # Carpeta actual
directorio_inicial1 = 'D:\sentirobots\Datasets\Base de Datos Unificada\Validation\Rostros MELD' # Carpeta actual
#directorio_inicial1 = 'D:\sentirobots\Datasets\MELD\Caras MELD\Test cropped images-20230410T143523Z-001\Test cropped images\Chandler'

pasa=[]
archivos1 = []
carpeta_base1=[]
puntos1=[]
filtro1=[]
file1=[]
b=[]
co=[]
e=[]
f=[]
d=[]
h=[]
w=[]

for raiz, carpetas, archivos_en_carpeta in os.walk(directorio_inicial1):
    for archivo in archivos_en_carpeta:
        if os.path.isfile(os.path.join(raiz, archivo)):
            ruta_completa1 = os.path.join(raiz, archivo)
            archivos1.append(ruta_completa1)
            b.append(evaluate_brillo(ruta_completa1))
            co.append(evaluate_contraste(ruta_completa1))
            e.append(evaluate_entropia(ruta_completa1))
            f.append(detect_faces(ruta_completa1))
            d.append(difuminacion(ruta_completa1))
            w.append(evaluate_resolucionW(ruta_completa1))
            h.append(evaluate_resolucionH(ruta_completa1))
            if evaluate_image_qualitys(ruta_completa1):
                pasa.append(1)
                if evaluate_brillo(ruta_completa1)>230 and evaluate_contraste(ruta_completa1)<20:
                    pasa[-1] = 2
                    print(ruta_completa1)
                    print("Transformacion brillo contraste o entropia")
                    print("brillo")
                    print(evaluate_brillo(ruta_completa1))
                    print("contraste")
                    print(evaluate_contraste(ruta_completa1))
                    print("entropia")
                    print(evaluate_entropia(ruta_completa1))
            else:
                pasa.append(0)
                if len(archivos1) > 1:
                    carpeta_anterior = os.path.dirname(archivos1[-2])
                    carpeta_actual = os.path.dirname(archivos1[-1])
                    a1 = os.path.join(archivos1[-2])
                    a2 = os.path.join(archivos1[-1])
                    if carpeta_anterior == carpeta_actual:
                        if pasa[-2] >= 1:
                            #PONER EL HOG CON LA SIMILITUD DEL COSENO
                            if (sim_coseno(a1, a2)>0.9 and difuminacion(ruta_completa1)>4.5 and detect_faces(ruta_completa1)>0) or (detect_faces(ruta_completa1)>9 and sim_coseno(a1, a2)>0.9) or (detect_faces(ruta_completa1)>5 and sim_coseno(a1, a2)>0.9 and difuminacion(ruta_completa1)>5 and evaluate_contraste(ruta_completa1)>4 and evaluate_brillo(ruta_completa1)>14 and evaluate_entropia(ruta_completa1)>4):
                                #print(a2)
                                #print("Transformacion")
                                pasa[-1] = 2
            #detectar_rostros_hog(ruta_completa1)

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\860103659.py:61: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\860103659.py:61: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:15: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in v

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:15: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:15: FutureWarning: `multichannel` is a deprecated argument name for `ho

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:15: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\860103659.py:61: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\860103659.py:61: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filte

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:15: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\860103659.py:61: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\860103659.py:61: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filte

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:15: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\860103659.py:61: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\860103659.py:61: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filte

...

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:15: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:15: FutureWarning: `multichannel` is a deprecated argument name for `ho

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:15: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\860103659.py:61: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\860103659.py:61: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filte

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:15: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:15: FutureWarning: `multichannel` is a deprecated argument name for `ho

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:15: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\860103659.py:61: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\860103659.py:61: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filte

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\635633044.py:15: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\860103659.py:61: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_15148\860103659.py:61: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filte

In [11]:
#Real
archiv={"RutaArchivo": archivos1}
df= pd.DataFrame(archiv)
df["Filtro"] = pasa
df["resolucion H"] = h
df["resolucion W"] = w
df["brillo"] = b
df["contraste"] = co
df["entropia"] = e
df["p rostro"] = f
df["difumin"] = d
df

,RutaArchivo,Filtro,resolucion H,resolucion W,brillo,contraste,entropia,p rostro,difumin
0,D:\sentirobots\Datasets\Base de Datos Unificad...,1,134,93,145.761114,37.182279,5.838623,20,37.182279
1,D:\sentirobots\Datasets\Base de Datos Unificad...,1,134,93,145.758065,37.222145,5.839069,20,37.222145
2,D:\sentirobots\Datasets\Base de Datos Unificad...,1,139,101,141.853195,103.177651,6.135112,60,103.177651
3,D:\sentirobots\Datasets\Base de Datos Unificad...,1,139,103,140.434937,103.228359,6.145396,66,103.228359
4,D:\sentirobots\Datasets\Base de Datos Unificad...,1,141,104,139.190330,134.828707,6.161817,81,134.828707
...,...,...,...,...,...,...,...,...,...
39061,D:\sentirobots\Datasets\Base de Datos Unificad...,1,225,169,103.560342,23.885344,4.840806,46,23.885344
39062,D:\sentirobots\Datasets\Base de Datos Unificad...,1,134,154,103.745493,81.764298,5.149011,39,81.764298
39063,D:\sentirobots\Datasets\Base de Datos Unificad...,1,132,152,104.882376,77.629335,5.137732,32,77.629335
39064,D:\sentirobots\Datasets\Base de Datos Unificad...,1,157,155,102.342141,73.058351,5.159123,49,73.058351


In [12]:
condicion0 = df['resolucion H'] < 64
condicion1 = df['resolucion W'] < 64
condicion2 = df['brillo'] < 14
condicion3 = df['contraste'] < 3.5
condicion4 = df['entropia'] < 3
condicion5 = df['p rostro'] <= 3
condicion6 = df['difumin'] <= 5.7

df['Falla en'] = pd.concat([condicion0, condicion1, condicion2, condicion3, condicion4, condicion5, condicion6], axis=1).sum(axis=1)
df

,RutaArchivo,Filtro,resolucion H,resolucion W,brillo,contraste,entropia,p rostro,difumin,Falla en
0,D:\sentirobots\Datasets\Base de Datos Unificad...,1,134,93,145.761114,37.182279,5.838623,20,37.182279,0
1,D:\sentirobots\Datasets\Base de Datos Unificad...,1,134,93,145.758065,37.222145,5.839069,20,37.222145,0
2,D:\sentirobots\Datasets\Base de Datos Unificad...,1,139,101,141.853195,103.177651,6.135112,60,103.177651,0
3,D:\sentirobots\Datasets\Base de Datos Unificad...,1,139,103,140.434937,103.228359,6.145396,66,103.228359,0
4,D:\sentirobots\Datasets\Base de Datos Unificad...,1,141,104,139.190330,134.828707,6.161817,81,134.828707,0
...,...,...,...,...,...,...,...,...,...,...
39061,D:\sentirobots\Datasets\Base de Datos Unificad...,1,225,169,103.560342,23.885344,4.840806,46,23.885344,0
39062,D:\sentirobots\Datasets\Base de Datos Unificad...,1,134,154,103.745493,81.764298,5.149011,39,81.764298,0
39063,D:\sentirobots\Datasets\Base de Datos Unificad...,1,132,152,104.882376,77.629335,5.137732,32,77.629335,0
39064,D:\sentirobots\Datasets\Base de Datos Unificad...,1,157,155,102.342141,73.058351,5.159123,49,73.058351,0


In [13]:
df.describe()

,Filtro,resolucion H,resolucion W,brillo,contraste,entropia,p rostro,difumin,Falla en
count,39066.000000,39066.000000,39066.000000,39066.000000,39066.000000,39066.000000,39066.000000,39066.000000,39066.000000
mean,0.928378,165.481288,123.000512,111.826506,62.292215,5.457280,39.390979,62.292215,0.114959
std,0.287795,47.271807,34.361303,22.326708,67.908040,0.392797,18.687690,67.908040,0.409497
min,0.000000,30.000000,14.000000,15.453395,2.609010,2.971824,0.000000,2.609010,0.000000
25%,1.000000,137.000000,105.000000,98.742796,29.551607,5.222349,27.000000,29.551607,0.000000
50%,1.000000,173.000000,129.000000,113.808558,45.094223,5.470274,37.000000,45.094223,0.000000
75%,1.000000,197.000000,144.000000,126.507710,69.535350,5.705554,49.000000,69.535350,0.000000
max,2.000000,399.000000,274.000000,206.579663,2081.629106,7.089142,238.000000,2081.629106,4.000000


In [14]:
cont = df['Filtro'].value_counts()
print(cont)

1    35630
0     3117
2      319
Name: Filtro, dtype: int64


In [15]:
conte = df['Falla en'].value_counts()
print(conte)

0    35630
1     2602
2      625
3      197
4       12
Name: Falla en, dtype: int64


In [16]:
df_fallas = df[df['Filtro'] == 0].copy()
df_buenas = df[df['Filtro'] == 1].copy()
df_recupera2 = df[df['Filtro'] == 2].copy()
df_perfe = df[df['Filtro'] >= 1].copy()

In [17]:
df_fallas

,RutaArchivo,Filtro,resolucion H,resolucion W,brillo,contraste,entropia,p rostro,difumin,Falla en
37,D:\sentirobots\Datasets\Base de Datos Unificad...,0,61,68,92.959016,51.898126,5.551311,14,51.898126,1
74,D:\sentirobots\Datasets\Base de Datos Unificad...,0,77,59,78.088708,68.445277,5.791165,13,68.445277,1
75,D:\sentirobots\Datasets\Base de Datos Unificad...,0,79,61,77.461299,49.146851,5.738230,10,49.146851,1
76,D:\sentirobots\Datasets\Base de Datos Unificad...,0,77,62,77.471722,62.814116,5.744143,10,62.814116,1
77,D:\sentirobots\Datasets\Base de Datos Unificad...,0,78,59,79.966754,43.372274,5.762653,15,43.372274,1
...,...,...,...,...,...,...,...,...,...,...
38808,D:\sentirobots\Datasets\Base de Datos Unificad...,0,92,53,93.650943,43.093106,5.567118,23,43.093106,1
38810,D:\sentirobots\Datasets\Base de Datos Unificad...,0,95,49,105.916864,35.346191,5.319260,16,35.346191,1
38811,D:\sentirobots\Datasets\Base de Datos Unificad...,0,87,49,108.024161,34.183539,5.294768,15,34.183539,1
38812,D:\sentirobots\Datasets\Base de Datos Unificad...,0,94,49,104.367781,29.149132,5.298548,17,29.149132,1


In [18]:
contefalla = df_fallas['Falla en'].value_counts()
print(contefalla)

1    2310
2     605
3     190
4      12
Name: Falla en, dtype: int64


In [19]:
df_fallas.describe()

,Filtro,resolucion H,resolucion W,brillo,contraste,entropia,p rostro,difumin,Falla en
count,3117.0,3117.000000,3117.000000,3117.000000,3117.000000,3117.000000,3117.000000,3117.000000,3117.000000
mean,0.0,77.414501,55.561758,103.516637,154.053694,5.667548,20.467437,154.053694,1.327559
std,0.0,26.460002,21.328121,28.960793,125.498370,0.710959,13.768207,125.498370,0.604477
min,0.0,30.000000,14.000000,15.453395,2.609010,2.971824,0.000000,2.609010,1.000000
25%,0.0,68.000000,48.000000,87.392251,59.580718,5.411707,9.000000,59.580718,1.000000
50%,0.0,74.000000,54.000000,104.968710,136.201235,5.872458,22.000000,136.201235,1.000000
75%,0.0,82.000000,60.000000,120.950427,224.445470,6.147040,30.000000,224.445470,2.000000
max,0.0,358.000000,271.000000,193.678175,2081.629106,7.089142,67.000000,2081.629106,4.000000


In [20]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(df_fallas['RutaArchivo']).tolist()

# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()

#     # Filtrar los registros correspondientes en el DataFrame
#     registros = df_fallas.loc[df_fallas['RutaArchivo'] == ruta]
#     # Mostrar los registros en la consola
#     print(registros)

In [21]:
df_buenas

,RutaArchivo,Filtro,resolucion H,resolucion W,brillo,contraste,entropia,p rostro,difumin,Falla en
0,D:\sentirobots\Datasets\Base de Datos Unificad...,1,134,93,145.761114,37.182279,5.838623,20,37.182279,0
1,D:\sentirobots\Datasets\Base de Datos Unificad...,1,134,93,145.758065,37.222145,5.839069,20,37.222145,0
2,D:\sentirobots\Datasets\Base de Datos Unificad...,1,139,101,141.853195,103.177651,6.135112,60,103.177651,0
3,D:\sentirobots\Datasets\Base de Datos Unificad...,1,139,103,140.434937,103.228359,6.145396,66,103.228359,0
4,D:\sentirobots\Datasets\Base de Datos Unificad...,1,141,104,139.190330,134.828707,6.161817,81,134.828707,0
...,...,...,...,...,...,...,...,...,...,...
39061,D:\sentirobots\Datasets\Base de Datos Unificad...,1,225,169,103.560342,23.885344,4.840806,46,23.885344,0
39062,D:\sentirobots\Datasets\Base de Datos Unificad...,1,134,154,103.745493,81.764298,5.149011,39,81.764298,0
39063,D:\sentirobots\Datasets\Base de Datos Unificad...,1,132,152,104.882376,77.629335,5.137732,32,77.629335,0
39064,D:\sentirobots\Datasets\Base de Datos Unificad...,1,157,155,102.342141,73.058351,5.159123,49,73.058351,0


In [22]:
contebuena = df_buenas['Falla en'].value_counts()
print(contebuena)

0    35630
Name: Falla en, dtype: int64


In [23]:
df_buenas.describe()

,Filtro,resolucion H,resolucion W,brillo,contraste,entropia,p rostro,difumin,Falla en
count,35630.0,35630.000000,35630.000000,35630.000000,35630.000000,35630.000000,35630.000000,35630.000000,35630.0
mean,1.0,173.969436,129.472018,112.516069,53.190516,5.436219,41.194050,53.190516,0.0
std,0.0,39.570752,27.857156,21.442460,48.037722,0.343553,18.114349,48.037722,0.0
min,1.0,64.000000,64.000000,29.448949,5.708418,3.525550,4.000000,5.708418,0.0
25%,1.0,152.000000,113.000000,99.623514,29.194695,5.215472,29.000000,29.194695,0.0
50%,1.0,178.000000,131.000000,114.373650,43.226008,5.448976,39.000000,43.226008,0.0
75%,1.0,199.000000,146.000000,126.897097,63.619295,5.669176,50.000000,63.619295,0.0
max,1.0,399.000000,274.000000,206.579663,1096.410253,6.734985,238.000000,1096.410253,0.0


In [24]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(df_buenas['RutaArchivo']).tolist()

# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()

#     # Filtrar los registros correspondientes en el DataFrame
#     registros = df_buenas.loc[df_buenas['RutaArchivo'] == ruta]
#     # Mostrar los registros en la consola
#     print(registros)

In [25]:
df_recupera2

,RutaArchivo,Filtro,resolucion H,resolucion W,brillo,contraste,entropia,p rostro,difumin,Falla en
1550,D:\sentirobots\Datasets\Base de Datos Unificad...,2,72,55,119.838384,198.729512,6.120348,34,198.729512,1
1551,D:\sentirobots\Datasets\Base de Datos Unificad...,2,73,55,119.367372,190.691357,6.076036,32,190.691357,1
1552,D:\sentirobots\Datasets\Base de Datos Unificad...,2,73,56,119.832926,215.329444,6.089986,30,215.329444,1
1553,D:\sentirobots\Datasets\Base de Datos Unificad...,2,74,55,119.514251,184.143477,6.083194,32,184.143477,1
1554,D:\sentirobots\Datasets\Base de Datos Unificad...,2,73,54,119.872146,191.989175,6.042453,29,191.989175,1
...,...,...,...,...,...,...,...,...,...,...
38826,D:\sentirobots\Datasets\Base de Datos Unificad...,2,73,62,100.778171,68.331270,5.244880,21,68.331270,1
38827,D:\sentirobots\Datasets\Base de Datos Unificad...,2,41,45,102.382114,809.753116,6.491350,27,809.753116,2
38828,D:\sentirobots\Datasets\Base de Datos Unificad...,2,36,47,102.271868,908.707505,6.566161,21,908.707505,2
38829,D:\sentirobots\Datasets\Base de Datos Unificad...,2,40,46,100.691304,922.870887,6.549167,20,922.870887,2


In [26]:
conterecu = df_recupera2['Falla en'].value_counts()
print(conterecu)

1    292
2     20
3      7
Name: Falla en, dtype: int64


In [27]:
df_recupera2.describe()

,Filtro,resolucion H,resolucion W,brillo,contraste,entropia,p rostro,difumin,Falla en
count,319.0,319.000000,319.000000,319.000000,319.000000,319.000000,319.000000,319.000000,319.000000
mean,2.0,77.931034,59.134796,116.004345,182.270294,5.755116,22.905956,182.270294,1.106583
std,0.0,20.797805,16.664013,27.500715,217.651249,0.452557,11.536813,217.651249,0.373561
min,2.0,36.000000,25.000000,55.834493,5.393010,4.014906,1.000000,5.393010,1.000000
25%,2.0,72.500000,54.000000,98.693773,80.898537,5.545979,16.000000,80.898537,1.000000
50%,2.0,76.000000,58.000000,108.205144,120.242534,5.911771,23.000000,120.242534,1.000000
75%,2.0,81.000000,62.000000,139.882542,205.877504,6.029176,30.000000,205.877504,1.000000
max,2.0,337.000000,263.000000,170.294503,1639.138009,6.566161,54.000000,1639.138009,3.000000


In [28]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(df_recupera2['RutaArchivo']).tolist()

# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()

#     # Filtrar los registros correspondientes en el DataFrame
#     registros = df_recupera2.loc[df_recupera2['RutaArchivo'] == ruta]
#     # Mostrar los registros en la consola
#     print(registros)

In [29]:
df_perfe

,RutaArchivo,Filtro,resolucion H,resolucion W,brillo,contraste,entropia,p rostro,difumin,Falla en
0,D:\sentirobots\Datasets\Base de Datos Unificad...,1,134,93,145.761114,37.182279,5.838623,20,37.182279,0
1,D:\sentirobots\Datasets\Base de Datos Unificad...,1,134,93,145.758065,37.222145,5.839069,20,37.222145,0
2,D:\sentirobots\Datasets\Base de Datos Unificad...,1,139,101,141.853195,103.177651,6.135112,60,103.177651,0
3,D:\sentirobots\Datasets\Base de Datos Unificad...,1,139,103,140.434937,103.228359,6.145396,66,103.228359,0
4,D:\sentirobots\Datasets\Base de Datos Unificad...,1,141,104,139.190330,134.828707,6.161817,81,134.828707,0
...,...,...,...,...,...,...,...,...,...,...
39061,D:\sentirobots\Datasets\Base de Datos Unificad...,1,225,169,103.560342,23.885344,4.840806,46,23.885344,0
39062,D:\sentirobots\Datasets\Base de Datos Unificad...,1,134,154,103.745493,81.764298,5.149011,39,81.764298,0
39063,D:\sentirobots\Datasets\Base de Datos Unificad...,1,132,152,104.882376,77.629335,5.137732,32,77.629335,0
39064,D:\sentirobots\Datasets\Base de Datos Unificad...,1,157,155,102.342141,73.058351,5.159123,49,73.058351,0


In [30]:
conteperfe = df_perfe['Falla en'].value_counts()
print(conteperfe)

0    35630
1      292
2       20
3        7
Name: Falla en, dtype: int64


In [31]:
df_perfe.describe()

,Filtro,resolucion H,resolucion W,brillo,contraste,entropia,p rostro,difumin,Falla en
count,35949.000000,35949.000000,35949.000000,35949.000000,35949.000000,35949.000000,35949.000000,35949.000000,35949.000000
mean,1.008874,173.117222,128.847868,112.547022,54.335929,5.439048,41.031767,54.335929,0.009819
std,0.093783,40.458583,28.550025,21.505727,53.411098,0.345959,18.147639,53.411098,0.109565
min,1.000000,36.000000,25.000000,29.448949,5.393010,3.525550,1.000000,5.393010,0.000000
25%,1.000000,151.000000,112.000000,99.600009,29.278567,5.217040,29.000000,29.278567,0.000000
50%,1.000000,177.000000,131.000000,114.349788,43.500026,5.451399,38.000000,43.500026,0.000000
75%,1.000000,199.000000,146.000000,126.916642,64.236544,5.673046,50.000000,64.236544,0.000000
max,2.000000,399.000000,274.000000,206.579663,1639.138009,6.734985,238.000000,1639.138009,3.000000


In [33]:
df.to_csv(r'D:\sentirobots\Datasets\AffWild2\MELD-Face-Val.csv', index=False)

print("El DataFrame ha sido guardado en un archivo CSV.")

El DataFrame ha sido guardado en un archivo CSV.


In [ ]:
###############################################################################

In [ ]:
# def ver_puntos_cara(lista):
#     if len(lista) > 0:
#         ultimo_elemento = lista[-1]  # Obtener el último elemento de la lista
#         if ultimo_elemento is not None and ultimo_elemento > 0:
#             return True
#     return False

In [ ]:
# #para mostrar las graficas HOG de rostros

# import matplotlib.pyplot as plt
# import cv2
# from skimage.feature import hog
# from skimage import exposure

# def detectar_rostros_hog(ruta_imagen):
#     # Cargar la imagen utilizando la ruta proporcionada
#     image = cv2.imread(ruta_imagen)

#     # Convertir la imagen a RGB (si está en formato BGR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#     # Calcular el descriptor HOG
#     fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
#                         cells_per_block=(1, 1), visualize=True, multichannel=True, channel_axis=2)

#     # Crear una figura con dos subfiguras para mostrar la imagen original y el HOG
#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

#     ax1.axis('off')
#     ax1.imshow(image, cmap=plt.cm.gray)
#     ax1.set_title('Imagen original')

#     # Reescalar el histograma para una mejor visualización
#     hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

#     ax2.axis('off')
#     ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
#     ax2.set_title('Histograma de gradientes orientados (HOG)')

#     plt.show()

In [ ]:
# #DF PARA FALLAS
# ima={"RutaArchivo": imagenes}
# dfallas= pd.DataFrame(ima)
# dfallas["Carpeta base"] = carpeta_base1
# dfallas["Archivo"] = n1
# dfallas["Tipo de falla"] = lista_fallas
# dfallas["puntos"] = puntosdet
# dfallas["brillo"] = brillo
# dfallas["contraste"] = contr
# dfallas["resolucion"] = reso
# dfallas["entropia"] = entro
# dfallas

In [ ]:
# conteo = dfallas['Tipo de falla'].value_counts()
# print(conteo)

In [ ]:
# df_agrupado = dfallas.groupby(['Carpeta base', 'Archivo']).agg(lambda x: x.fillna(method='ffill').iloc[-1])
# df_agrupado

In [ ]:
# df_agrupado['no nulos'] = df_agrupado[['puntos', 'brillo', 'contraste', 'resolucion', 'entropia']].notnull().sum(axis=1)
# df_agrupado

In [ ]:
# conteo = df_agrupado['Tipo de falla'].value_counts()
# print(conteo)

In [ ]:
# conte = df_agrupado['no nulos'].value_counts()
# print(conte)

In [ ]:
# df_agrupado.describe()

In [ ]:
# dfresolucion = df_agrupado[pd.to_numeric(df_agrupado['resolucion'], errors='coerce').notnull()]
# dfresolucion

In [ ]:
# conteo0 = dfresolucion['resolucion'].value_counts()
# print(conteo0)

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(dfresolucion['RutaArchivo']).tolist()
# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()

In [ ]:
# dfpuntos = df_agrupado[pd.to_numeric(df_agrupado['puntos'], errors='coerce').notnull()]
# dfpuntos

In [ ]:
# dfpuntos.describe()

In [ ]:
# conteo1 = dfpuntos['puntos'].value_counts()
# print(conteo1)

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(dfpuntos['RutaArchivo']).tolist()
# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(dfpuntos['RutaArchivo']).tolist()

# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()

#     # Filtrar los registros correspondientes en el DataFrame
#     registros = dfpuntos.loc[dfpuntos['RutaArchivo'] == ruta]
#     # Mostrar los registros en la consola
#     print(registros)

In [ ]:
# dfbrillo = df_agrupado[pd.to_numeric(df_agrupado['brillo'], errors='coerce').notnull()]
# dfbrillo

In [ ]:
# dfbrillo.describe()

In [ ]:
# from collections import Counter
# valores_unicos = dfbrillo['brillo'].round()
# valores_unicos = sorted(valores_unicos, reverse=True)
# conteo = Counter(valores_unicos)
# conteo

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(dfbrillo['RutaArchivo']).tolist()
# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()

    
#     #Descartar bajo 14.5 y entran todas las caras

In [ ]:
# dfcontraste = df_agrupado[pd.to_numeric(df_agrupado['contraste'], errors='coerce').notnull()]
# dfcontraste

In [ ]:
# dfcontraste.describe()

In [ ]:
# valores_unicos = dfcontraste['contraste'].unique()
# valores_unicos = sorted(valores_unicos, reverse=True)
# valores_unicos

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(dfcontraste['RutaArchivo']).tolist()
# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()
    
#     #3.5 hacia arriba es aceptable

In [ ]:
# dfentropia = df_agrupado[pd.to_numeric(df_agrupado['entropia'], errors='coerce').notnull()]
# dfentropia

In [ ]:
# dfentropia.describe()

In [ ]:
# conteo5 = dfentropia['entropia'].value_counts()
# print(conteo5)

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(dfentropia['RutaArchivo']).tolist()
# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()
#     #esta bien como está

In [ ]:
####################################################################################################################

In [ ]:
# #PARA LA DETECCION REAL
# import os
# import pandas as pd
# import numpy as np
# file=[]
# #directorio_inicial = 'D:\sentirobots\Datasets\AFEW\Datos\Train_AFEW\AlignedFaces_LBPTOP_Points\AlignedFaces_LBPTOP_Points\Ejercicio entropia' # Carpeta actual
# directorio_inicial = 'D:\sentirobots\Datasets\AFEW\Datos\Train_AFEW\AlignedFaces_LBPTOP_Points\AlignedFaces_LBPTOP_Points\Faces' # Carpeta actual
# # Creamos una lista vacía para almacenar los nombres de archivo
# archivos = []
# calidad=[]
# carpeta_base=[]
# puntos=[]
# keypo=[]
# filtro=[]
# # Recorremos todos los archivos y carpetas a partir del directorio inicial
# for raiz, carpetas, archivos_en_carpeta in os.walk(directorio_inicial):
#     # Iteramos por cada archivo en la carpeta actual y lo agregamos a la lista de archivos
#     for archivo in archivos_en_carpeta:
#         # Verificamos si el elemento actual es un archivo
#         file.append(archivo)
#         if os.path.isfile(os.path.join(raiz, archivo)):
#             ruta_completa = os.path.join(raiz, archivo)
#             carpet, archi = os.path.split(ruta_completa)
#             c= os.path.basename(carpet)
#             carpeta_base.append(c)
#             archivos.append(ruta_completa)
#             if evaluate_image_quality(ruta_completa): # si la función retorna True
#                 cal=1
#                 calidad.append(cal) # agregamos un 1 a la lista
#             else: # si la función retorna False
#                 cal=0
#                 calidad.append(cal) # agregamos un 0 a la lista
#             if detect_faces(ruta_completa)>=5:
#                 p=1
#                 puntos.append(p)
#             else:
#                 p=0
#                 puntos.append(p)
#             if detect_faces(ruta_completa)>=5 and cal==1 or detect_faces(ruta_completa)>=5 and cal==0:
#                 filtro.append(1)
#             if  detect_faces(ruta_completa)<5 and cal==0 or detect_faces(ruta_completa)<5 and cal==1:
#                 filtro.append(0)
#             #print(len(filtro))
#             #print(len(puntos))
#             keypo.append(detect_faces(ruta_completa))

In [ ]:
# archiv={"RutaArchivo": archivos}
# df= pd.DataFrame(archiv)
# df["Carpeta final"] = carpeta_base
# df["Archivo"] = file
# df["Calidad"] = calidad
# df["Puntos"] = puntos
# df["KeyPoints"] = keypo
# df["Filtro"] = filtro

In [ ]:
# df

In [ ]:
# # Creamos un DataFrame vacío donde agregaremos las filas que cumplan la condición
# dfpuntoscriticos = pd.DataFrame(columns=['Filtro'])

# # Iteramos sobre cada fila del DataFrame original
# for index, row in df.iterrows():
#     # Comparamos si las dos columnas son iguales
#     #if row['Calidad'] != row['Puntos']:
#     if row['Filtro'] == 0:
#         # Si son iguales, agregamos la fila completa al DataFrame de resultado
#         dfpuntoscriticos = pd.concat([dfpuntoscriticos, row.to_frame().T], ignore_index=True)

# # Imprimimos el DataFrame de resultado
# dfpuntoscriticos

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(dfpuntoscriticos['RutaArchivo']).tolist()
# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()

In [ ]:
# df.describe()

In [ ]:
# import os
# import cv2
# import numpy as np
# #NO USAR PORQUE ESTARÍA ASUMIENDO QUE SOLAMENTE HAY UNA PERSONA EN CADA CARPETA, SE PUEDE USAR PARA CONTAR CADA VEZ QUE HAY UN IDA Y VUELTA EN CAMARA O ESTIMAR CUANTAS PERSONAS HAY POR CARPETA. 
# def listar_archivos(directorio, archivos=None, coseno=[], usar=[]):
#     # Si no se proporcionó una lista, crear una vacía
#     if archivos is None:
#         archivos = []
#     # Inicializar la ruta de img1 en None
#     img1_ruta = None
#     # Recorrer todas las carpetas y archivos del directorio
#     for nombre_archivo in os.listdir(directorio):
#         # Unir el directorio y el nombre del archivo para obtener la ruta completa
#         ruta_archivo = os.path.join(directorio, nombre_archivo)
#         # Si es un archivo, guardar su ruta y nombre en la lista y comparar con el archivo anterior
#         if os.path.isfile(ruta_archivo):
#             archivos.append(ruta_archivo)
#             # Comprobar si el archivo anterior y el actual están en la misma carpeta
#             if len(archivos) > 1:
#                 carpeta_anterior = os.path.dirname(archivos[-2])
#                 carpeta_actual = os.path.dirname(archivos[-1])
#                 a1 = os.path.join(archivos[-2])
#                 a2 = os.path.join(archivos[-1])
#                 if carpeta_anterior == carpeta_actual:
#                     # Aquí puedes poner la acción que quieres que se haga
#                     if img1_ruta is None or cos_sim >= 0.85:
#                         img1_ruta = a1
#                     comparar = img1_ruta
#                     img1 = cv2.imread(comparar, cv2.IMREAD_GRAYSCALE)
#                     img2 = cv2.imread(a2, cv2.IMREAD_GRAYSCALE)

#                     # Aplicar un resize a las imágenes new_width=128, new_height=128
#                     img1 = cv2.resize(img1, (128, 128))
#                     img2 = cv2.resize(img2, (128, 128))

#                     # Aplanar las imágenes y normalizarlas
#                     img1 = img1.flatten() / 255.0
#                     img2 = img2.flatten() / 255.0

#                     # Calcular el producto punto entre ambas imágenes y las normas de cada una
#                     dot_product = np.dot(img1, img2)
#                     norm_img1 = np.linalg.norm(img1)
#                     norm_img2 = np.linalg.norm(img2)

#                     # Calcular la similitud del coseno entre ambas imágenes
#                     cos_sim = dot_product / (norm_img1 * norm_img2)
#                     print(cos_sim)
#                     coseno.append(cos_sim)
#                     if cos_sim<0.85:#0 es que no se puede usar
#                         usar.append(0)
#                     else:#1 es que se puede usar
#                         usar.append(1)
#                 else:
#                     coseno.append(1)
#                     usar.append(1)
#                     print(ruta_archivo)
#                     # Restablecer la ruta de img1 a None para la próxima carpeta
#                     img1_ruta = None
#             else:
#                 coseno.append(1)
#                 usar.append(1)
#                 print(ruta_archivo)
#                 # Restablecer la ruta de img1 a None para la próxima carpeta
#                 img1_ruta = None
#         # Si es una carpeta, recorrer sus archivos y guardarlos en la lista
#         elif os.path.isdir(ruta_archivo):
#             listar_archivos(ruta_archivo, archivos=archivos, coseno=coseno, usar=usar)
#         print(len(archivos))
#         print(img1_ruta)
#         print(len(coseno))
#     return archivos, coseno, usar

# # Ejemplo de uso
# directorio_inicial = 'D:\sentirobots\Datasets\AFEW\Datos\Train_AFEW\AlignedFaces_LBPTOP_Points\AlignedFaces_LBPTOP_Points\Faces' # Carpeta actual
# archivos = listar_archivos(directorio_inicial)


In [ ]:
# #evaluate_image_quality
# import cv2
# from skimage import io, img_as_float
# from skimage.filters import rank
# from skimage.morphology import disk
# import numpy as np
# def evaluate_image_quality(a):
#     # Cargar la imagen
#     #print(a)
#     img = cv2.imread(a)
#     # Verificar la resolución de la imagen
#     height, width, channels = img.shape
#     if height < 100 or width < 100:
#         #print("Falla hw")
#         return False

#     # Calcular la iluminación promedio de la imagen
#     hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#     h, s, v = cv2.split(hsv)
#     mean_brightness = v.mean()
#     if mean_brightness < 14.5:
#         #print("Falla brillo")
#         #print(mean_brightness)
#         return False

#     # Calcular el contraste promedio de la imagen
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     mean_contrast = cv2.Laplacian(gray, cv2.CV_64F).var()
#     if mean_contrast < 3.5:
#         #print("mean contraste")
#         #print(mean_contrast)
#         #print("Falla contraste")
#         return False
    
#     # Calcular la entropía de la imagen
#     im1 = img_as_float(io.imread(a, as_gray=True))
#     radius = 15
#     selem = disk(radius)
#     entropy = rank.entropy(im1, selem=selem)
#     mean_entropy = np.mean(entropy)
#     # Evaluar entropia
#     if mean_entropy < 3:
#         #print("Falla entropia")
#         #print(mean_entropy)
#         return False
    
#     # Si todos los criterios se cumplen, la imagen es de buena calidad
#     return True 